## Capstone Project - The Battle of Neighborhoods (Week 2)
### Applied DataScience Capstone by IBM/Coursera 

### Table of contents
 


 #### Business Problem
 #### Data Description
 #### Methodology
 #### Analysis
 #### Results and Discussion
 #### Conclusion

#### Business Problem
In this project we will try to suggest a good location yo start Grocery Business in NC. The Main focus will be on opening an Indian Grocery store in NC, US.
As there are many Indian Grocery Stores in NC,The locations that are not surrounded by indan restaurants are Identified.
More focus is on finding out the locations which are surrounded by Indians. 
By Using Data Science techniques we will try to find out the best locations to start Grocery Store business where maximum profits are acheived.


#### Data Description

The very key things that will effect decission making are as follows:
The number of grocery stores present in the location.
The number of Business Units present in the location and distance between them.

Following data sources will be needed to extract/generate the required information:
The number of grocery stores present in the location will be obtained using Foursquare API.
The type of grocery stores in every neighborhood will be obtained using Foursquare API.
Coordinates will be obtained by using Google Maps API geocoding and reverse geocoding.

#### Loading Libraries

In [1]:
# Loading libraries

import numpy as np
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

from bs4 import BeautifulSoup

import geocoder 
import copy
import shapely.geometry
import pyproj
import math

print('Importing libraries completed.')

Importing libraries completed.


In [2]:
# Finding latitude & longitude of Locations 
col1 = ['Location Name','Latitude','Longitude']

loc = pd.DataFrame(columns = col1)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['Postal Code'] = ['00601','00602','00603','00606']
loc

,Location Name,Latitude,Longitude,Postal Code
0,Parkwoods,NaN,NaN,00601
1,Victoria Village,NaN,NaN,00602
2,Harbourfront,NaN,NaN,00603
3,Lawrence Heights,NaN,NaN,00606


In [3]:
google_api_key = 'masked'

def get_coordinates(api_key, address, verbrose=False):
    try:
        url = "https://gist.github.com/erichurst/7882666".format(address, api_key)
        response=requests.get(url).json()
        if verbose:
            print('https://gist.github.com/erichurst/7882666', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location']
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return[lat,lon]
    except:
        return [None, None]
a=0
for i in loc['Location Name']:
        lat, lon = get_coordinates(google_api_key, i)
        loc.iloc[a,2] = lat
        loc.iloc[a,3] = lon
        a= a +1
        
loc

col= ['Location Name','Zipcode','Latitude','Longitude']

loc= pd.DataFrame(columns = col)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['Zipcode'] = ['00601','00602','00603','00606']
loc['Latitude'] = ['18.180555','18.361945','18.455183','18.158345']
loc['Longitude'] = ['-66.749961','-67.175597','-67.119887','-66.932911']
loc


,Location Name,Zipcode,Latitude,Longitude
0,Parkwoods,00601,18.180555,-66.749961
1,Victoria Village,00602,18.361945,-67.175597
2,Harbourfront,00603,18.455183,-67.119887
3,Lawrence Heights,00606,18.158345,-66.932911


In [4]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj='utm',zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj='utm',zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [ ]:
import shapely.geometry
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Berlin center longitude={}, latitude={}'.format(NC[1], NC[0]))
x, y = lonlat_to_xy(NC[1], NC[0])
print('NC UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('NC longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check

NC longitude=13.4132147, latitude=52.5219184

NC center UTM X=392341.28017572395, Y=5820273.243274779

NC longitude=13.413214700000001, latitude=52.52191839999997

In [5]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import bs4
import lxml.html as lh

In [6]:
url = 'https://gist.github.com/erichurst/7882666'
req = requests.get(url)

In [7]:
soup = bs4.BeautifulSoup(req.text,'html5lib')

In [8]:
data = soup.select('.wikitable.sortable')
print(type(data))
print(len(data))

<class 'list'>
0


In [9]:
doc = lh.fromstring(req.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [10]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    

In [11]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        col[i][1].append(data)
        i+=1

In [12]:
# Dataframe creation form List

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)


In [ ]:
for i in range(0,211):
    df.iloc[i,2] = df.iloc[i,2].strip('\n')

In [15]:
df = df.reset_index(drop = True)

In [ ]:
for i in range(0,211):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]

In [ ]:
df.head(5)

	Postcode	Borough	 Neighbourhood
    
0	M3A	North York	     Parkwoods

1	M4A	North York	     Victoria Village

2	M5A	Downtown Toronto	Harbourfront

3	M5A	Downtown Toronto	Regent Park

4	M6A	North York	Lawrence Heights

In [ ]:
addresses[150:170]

['Morrisville 12, 2112 NC',
 'Cary 01, 2112 NC',
 'GreensBorrow 03, 2112 NC 3112 NC',
 'Winson 9, 3112 NC',
 'Salom 1, 3112 NC',]

In [ ]:
df_locations.to_pickle('./locations.pkl')

#### Foursquare 

In [19]:
CLIENT_ID = 'MS' 
CLIENT_SECRET = 'MS'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MS
CLIENT_SECRET:MS


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=6000, LIMIT=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
        
              name,
              lat,
              lng,
              v['venue']['name'],
              v['venue']['location']['lat'],
              v['venue']['location']['lng'],
              v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['postcode',
                                 'NeighborhoodLatitude',
                                 'NeighborhoodLongitude',
                                 'Venue',
                                 'VenueLatitude',
                                 'VenueLongitude',
                                 'VenueCategory']
        return(nearby_venues)
                            

In [ ]:
loc= pd.DataFrame(columns = col)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['state'] = ['NC','NC','NC','NC']
loc

In [21]:
len(loc)

4

In [ ]:
import numpy as np

print('Total number of Grocery Stores:', len(Grocery Stores))
print('Total number of Indian Grocery Stores:', len(Indian Grocery Stores))
print('Percentage of Indian Grocery Stores: {:.2f}%'.format(len(Indian Grocery Stores) / len(Grocery Stores) * 100))
print('Average number of Grocery Stores:', np.array([len(r) for r in location_GroceryStores]).mean())

Total number of Grocery Stores: 200

Total number of Indian Grocery Stores: 150

Percentage of Indian Grocery Stores: 9%

Average number of Grocery Stores: 1.47810093

In [ ]:
print('List of Grocery stores')
print('-----------------------')
for r in list(store.values())[:10]:
    print(r)
print('...')
print('Total:', len(Stores))

List of Grocery stores

('72a498e349bf0e737e1', 'Indo Pak', 5.474363806181806, 1.4220184326, 'rl-ax-Srae 177, 10247 NC', 249, False, 52.75333317, 58144.355430137)
('e4b0f39fffdd0447', 'terei', 2.278495275, 1.3532529153, 'nbergplatz 2d, 10623 NC', 133, False, 386862.9315917266, 5806.191572046)
('8fa512462a6b490', 'nein ff', 5.6953, 3.5849, 'Unter der Linden 42, 10117 NC', 69, False, 390472.37417370133, 9762.151308152)
('514316eae4b080a105a5b4f5', 'ee stro', 52.534855836549994, 13.497241138618675, 'NC', 279, False, 398071.8391866421, 5821590.182515125)

...

Total: 41

In [ ]:
print('List of Indian Grocery stores')
print('---------------------------')
for r in list(Indian Grocery stores.values())[:10]:
    print(r)
print('...')
print('Total:', len(Indian Grocery stores))

List of Indian Grocery stores

('3f964a520e10327e3', 'meria inario', 5.482745, 1.537136029817, 'kystr. 3 (guststr.), 101 NC', 123, True, 397.4631, 00.632)
('eda2c7124a8f0', 'arella', 2.28238963, 3.2905663293, 'NC', 18, True, 380.140888, 821469.741)
('9c54b273', 'Ata risi', 2.364004, 1.34496, 'tstr. 5, 15 NC', 1, True, 33.7624, 99.2318)
('5f10', "Allo's cheria & Street", 5.4539, 3.94, 'tzstr. 2, NC', 40, True, 33.0626, 105.1504)
...

Total: 29

#### Methodology 


In this project let us find out the locations with less grocery Stores, main focus will be on the areas which has more indian grocery stores and more populated i.e density of indian population is more. 

Data Collected - Type of location that is good to start grocery store business and good location for more profits and found Indian Grocery stores through Foursquare categorization.

Next in Analysis phase shall calculate the population density of Indians in the NC regions and identify good locations to start the grocery store business.

Next we shall focus on the best location where grocery store business can be eshtablished and more profits can be acheived through Clustering.

#### Analysis

In [ ]:
loc_Store_count = [len(store) for store in loc_Store]

df_loc['Grocery stores in area'] = loc_Store_count

print('Average no of grocery stores:', np.array(loc_Store_count).mean())


Average no of grocery stores: 2.1209



In [ ]:
print('Average distance from other stores :', df_loc['Distance to Indian Grocery store'].mean())

Average distance from other stores: 32.0672

In [ ]:
Grocery Store_latlons = [[store[2], res[3]] for store in Grocery Store.values()]

Indian Grocery Store_latlons = [[res[2], res[3]] for res in Indian Grocery Store.values()]

In [ ]:
k = math.sqrt(3) / 2
x_step = 100
y_step = 100 * k 
r_min = ri_y - 2500

r_latitudes = []
r_longitudes = []
r_xs = []
r_ys = []
for i in range(0, int(51/k)):
    y = r_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = r_min + j * x_step + x_offset
        d = calc_xy_distance(r_center_x, r_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            r_latitudes.append(lat)
            r_longitudes.append(lon)
            r_xs.append(x)
            r_ys.append(y)

print(len(r_latitudes), 'Grocery stores in Neighbourhood.')

150 Grocery stores in Neighbourhood.

In [ ]:
store_count = np.array((df_r_locations['Stores nearby']<=2))
print('Locations with no more than two stores nearby:', store_count.sum())

store_distance = np.array(df_r_locations['Distance to Indian Grocery stores']>=400)
print('Locations with no Indian Grocery Stores:', store_distance.sum())

store_locations = np.logical_and(store_count, store_distance)
print('Locations with both conditions met:', store_locations.sum())

df_store_locations = df_r_locations[store_locations]

Locations with no more than two stores nearby: 40

Locations with no Indian Grocery Stores: 35

Locations with both conditions met: 20

In [ ]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_NC = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_NC)
HeatMap(Store_latlons).add_to(map_NC)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_NC)
folium.Marker(NC_center).add_to(map_NC)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_NC) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_NC)
folium.GeoJson(NC_boroughs, style_function=boroughs_style, name='geojson').add_to(map_NC)
map_NC

#### Results and Discussion 

From our Analysis we can say that there are good number of grocery stores in NC irrespective of type, there are grocery stores including chinese, Indian, American, Korean, Indo-Pak etc. These areas are coverd with different kinds of people comming from different parts of the globe.

There are few places like morrisville, cary, Raleigh, Durham, where there are great number of grocery stores of all types apart from this there are few areas in this regions where there are indians great in number and there is a probability of earning more profits if the gocery store business is eshtablished in this regions. 

Final Result is there are 4 regions containing largest number of new grocery store locations based on number of indians residing in that particular location.Purpose of this analysis was to only provide info on areas with more indian polpulation which are good to start grocery store business where there are less indian grocery stores with more returns in terms of profits are expected in return to investment.


#### Conclusion 

Main Purpose of this project was to identify the areas in NC which has more indian polulation density and less Indian grocery stores so that a grocery store business can be eshtablished and it would be an good investment option to the client to invest on his new business where more profits are acheived in return.By calculating grocery store density distribution from Foursquare data , Clustering techniques were used on those locations as part of further analysis.
